# Modélisation SIR:
Ci_dessous notre modélisation SIR sur les données mondiales de la covid 19.

## Données: 
Source de ce jeu de données : https://coronavirus.politologue.com 
Les données de ce fichier sont une compilation des données proposées par le CSSE (The Johns Hopkins University) https://gisanddata.maps.arcgis.com/apps/opsdashboard/index.html#/bda7594740fd40299423467b48e9ecf6 
Les données de ce fichier sont celles qui sont produites pour générer les graphiques sur  https://coronavirus.politologue.com

## Légende:
s: susceptible
i: infecté
r: guéris

In [37]:
# téléchargement des librairies

%matplotlib inline
import numpy as np
import  matplotlib.pyplot as plt
import  pandas as pd
import scipy
from scipy.integrate import odeint
from scipy.io import savemat, loadmat
import numpy.linalg as la
from matplotlib.lines import Line2D
import plotly.express as px
from scipy.optimize import minimize


In [4]:
# téléchargement des données

df_path=  ("C:/Users/Dell/Downloads/coronavirus.politologue.com-pays-2021-05-15.csv")
df = pd.read_csv(df_path, sep=";", encoding='utf-8',error_bad_lines=False)
df['Date'] = pd.to_datetime(df['Date'],format='%Y-%m-%d')

In [5]:
df

,Date,Pays,Infections,Deces,Guerisons,TauxDeces,TauxGuerison,TauxInfection
0,2021-05-15,Andorre,13510,127,13155,0.94,97.37,1.69
1,2021-05-15,Émirats Arabes Unis,544931,1629,525080,0.30,96.36,3.34
2,2021-05-15,Afghanistan,63355,2730,54663,4.31,86.28,9.41
3,2021-05-15,Antigua-et-Barbuda,1240,32,1179,2.58,95.08,2.34
4,2021-05-15,Albanie,131939,2427,122105,1.84,92.55,5.61
...,...,...,...,...,...,...,...,...
6288,2021-04-15,Mayotte,19757,168,2964,0.85,15.00,84.15
6289,2021-04-15,Afrique du Sud,1562931,53571,1488060,3.43,95.21,1.36
6290,2021-04-15,Zambie,90532,1230,88283,1.36,97.52,1.13
6291,2021-04-15,Zimbabwe,37422,1550,34961,4.14,93.42,2.43


In [25]:
# lisons les chiffres en calculant la moyenne mobile sur 3 jours sur toutes les colonnes

for x in ['Infections','Guerisons','Deces']:
    df[x+"_Av"] = df[x].rolling(window=100).mean()
df['Removed_Av'] = df['Guerisons_Av']+df['Deces_Av']

In [26]:
# On se concentre ici sur une periode de 7 jours

start_index = df[df['Infections_Av']>1000].index[0]
ndays = 7
V = df['Infections_Av'][start_index:start_index+ndays].to_numpy()
R = df['Removed_Av'][start_index:start_index+ndays].to_numpy()

In [30]:

def deriv(y, t, N, beta, gamma):
    S, I, R = y
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt

In [31]:

def model(V,R,N,beta,gamma):
    t = np.linspace(0,len(V),len(V))
    y0 = N-V[0]-R[0],V[0],R[0]
    S,I,R = odeint(deriv, y0, t, args=(N, beta, gamma)).T
    dV = np.diff(V)
    dI = np.diff(I+R)
    return np.linalg.norm(dV-dI)

In [32]:
# Afin de trouver la valeur optimale de beta, nous utilisons la méthode minimize de SciKit Learn

the_gamma = 1/30
def fit(V,R,N):
    res = scipy.optimize.minimize(lambda x:model(V,R,N,x,the_gamma),
                            x0=0.5,method='powell')
    return res.x

In [33]:
fit(V, R, 1200000)

C:\Users\Dell\Anaconda3\lib\site-packages\scipy\integrate\odepack.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


array([0.00115008])

In [34]:
import numpy as np
from scipy.optimize import minimize


In [35]:
fit(V, R, 120000000)

array([-0.00041341])

In [49]:
graphe_deces = px.line(df, x=df['Date'], y=df['Deces'], title="Deces au cours du temps")
graphe_deces

In [50]:
graphe_infections = px.line(df, x=df['Date'], y=df['Infections'], title="Infections au cours du temps")
graphe_infections

In [51]:
graphe_guerisons = px.line(df, x=df['Date'], y=df['Guerisons'], title="Guerisons au cours du temps")
graphe_guerisons

In [40]:
# filtrer les données de la France

df_fr = df[ df['Pays'] == 'France' ]
df_fr.head()

,Date,Pays,Infections,Deces,Guerisons,TauxDeces,TauxGuerison,TauxInfection,Infections_Av,Guerisons_Av,Deces_Av,Removed_Av
59,2021-05-15,France,5796528,106659,319867,1.84,5.52,92.64,NaN,NaN,NaN,NaN
346,2021-05-14,France,5796528,106659,319867,1.84,5.52,92.64,744546.51,537626.21,15408.20,553034.41
465,2021-05-13,France,5790059,106488,319094,1.84,5.51,92.65,787287.77,603802.43,18804.13,622606.56
752,2021-05-12,France,5770598,106357,318027,1.84,5.51,92.65,738591.69,531832.80,15304.47,547137.27
871,2021-05-11,France,5750568,106193,316537,1.85,5.50,92.65,780308.03,599208.12,18638.00,617846.12


## visualiser les données des infections, guérisons et décès

In [52]:
graphe_deces_fr = px.line(df_fr, x=df_fr['Date'], y=df_fr['Deces'], title="Deces au cours du temps en France")
graphe_deces_fr

In [54]:
graphe_infections_fr = px.line(df_fr, x=df_fr['Date'], y=df_fr['Infections'], title="Infections au cours du temps en France")
graphe_infections_fr

In [44]:
graphe_guerisons_fr = px.line(df_fr, x=df_fr['Date'], y=df_fr['Guerisons'],title="Guerisons au cours du temps en France")
graphe_guerisons_fr

In [45]:
graphe_deces_pays = pu.line(df, x=df['Pays'], y=df['Deces'], title="Deces par pays")
graphe_deces_pays

In [46]:
graphe_infections_pays = px.line(df, x=df['Pays'], y=df['Infections'], title="Infections par pays")
graphe_infections_pays

In [47]:
graphe_guerisons_pays = px.line(df, x=df['Pays'], y=df['Guerisons'], title="Guerisons par pays")
graphe_guerisons_pays